# <center>Fraud Detection

### The data provided is of all the customers who's insurnce is been claimed and in that customers whom claim is classified asfraud or not.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder

## Reading the Demographics data set (after removing null values)

In [2]:
# Demographics Data : These files consist of the demographic data of each customer
dfd = pd.read_csv("Demographics.csv",na_values= ["", "NA"])
dfd.drop("Unnamed: 0",axis=1,inplace=True)
print(dfd.shape)
dfd.head()

(28836, 9)


,CustomerID,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss
0,Cust10000,35,454776,MALE,JD,armed-forces,movies,56700,-48500
1,Cust10001,36,454776,MALE,JD,tech-support,cross-fit,70600,-48500
2,Cust10002,33,603260,MALE,JD,armed-forces,polo,66400,-63700
3,Cust10003,36,474848,MALE,JD,armed-forces,polo,47900,-73400
4,Cust10004,29,457942,FEMALE,High School,exec-managerial,dancing,0,-41500


In [3]:
# Missing values in the Demographics data set
dfd.isnull().sum().sum()

0

## Reading the policy claim data set

In [4]:
# Data of Claim : These files consist of the details about the insurance claim,that the customer applied for.
dfc = pd.read_csv("PolicyClaim.csv",na_values= [ "?", "-5", "MISSINGVALUE", "MISSEDDATA"])
print(dfc.shape)
dfc.head()

# index 16 and 17 is fishy -- check if they are fraud case

(28836, 21)


,CustomerID,TypeOfIncident,TypeOfCollission,SeverityOfIncident,AuthoritiesContacted,IncidentState,IncidentCity,IncidentAddress,IncidentTime,NumberOfVehicles,...,BodilyInjuries,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,Year,Month,days
0,Cust10001,Multi-vehicle Collision,Side Collision,Total Loss,Police,State7,City5,Location 1311,10.0,3,...,2,1.0,YES,61382.0,15560,5919,39903,2015,2,2
1,Cust10002,Single Vehicle Collision,Side Collision,Minor Damage,Other,State8,City6,Location 2081,22.0,1,...,2,3.0,NO,66755.0,11630,11630,43495,2015,1,15
2,Cust10003,Single Vehicle Collision,Side Collision,Minor Damage,Other,State9,City6,Location 2081,22.0,1,...,2,3.0,NO,66243.0,12003,12003,42237,2015,1,19
3,Cust10004,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State8,City6,Location 1695,10.0,1,...,2,1.0,YES,53544.0,8829,7234,37481,2015,1,9
4,Cust10005,Single Vehicle Collision,Rear Collision,Minor Damage,Fire,State7,City6,Location 1695,7.0,1,...,1,2.0,NoReport,53167.0,7818,8132,37217,2015,2,7


In [5]:
# Missing values in the policy claim dataframe
dfc.isnull().sum().sum()

0

### Here maybe the missing value itself maybe an information, so for some columns i have imputed a new value that indicates that value was missing and for property damage i have done model based imputation using KNN.

## Reading the Policy Information data set

In [6]:
# Policy Information: insurance policy information, connected to the claim with the insurance company.
dfp = pd.read_csv(r"C:\Users\Soorya\Documents\capstone-files\TrainData\Data\Train_Policy.csv",na_values=["-1", "MISSINGVAL","NA"])
print(dfp.shape)
dfp.head()

#Policy_CombinedSingleLimit -- seems to convert into percentage 
#InsuredRelationship -- maybe marital status

(28836, 10)


,InsurancePolicyNumber,CustomerLoyaltyPeriod,DateOfPolicyCoverage,InsurancePolicyState,Policy_CombinedSingleLimit,Policy_Deductible,PolicyAnnualPremium,UmbrellaLimit,InsuredRelationship,CustomerID
0,110122,328,2014-10-17,State3,250/500,1000,1406.91,0,husband,Cust1001
1,110125,256,1990-05-25,State1,250/500,2000,1415.74,6000000,unmarried,Cust1004
2,110126,228,2014-06-06,State1,500/1000,1000,1583.91,6000000,unmarried,Cust1005
3,110127,256,2006-10-12,State3,250/500,1000,1351.10,0,unmarried,Cust1006
4,110128,137,2000-06-04,State2,250/500,1000,1333.35,0,husband,Cust1007


In [7]:
dfp.isnull().sum().sum()

141

In [8]:
dfp['PolicyAnnualPremium'].median()

1266.44

In [9]:
# null value is replaced with median value

dfp['PolicyAnnualPremium'].fillna(dfp['PolicyAnnualPremium'].median(),inplace=True)

In [10]:
dfp.isnull().sum().sum()

0

## Reading the Vehicle Information data set

In [12]:
#Data of Vehicle : These files consist of the details about the Vehicle, connected to the policy. 
dfv = pd.read_csv("Vehicles_clean.csv",na_values=["???"])
dfv['VehicleModel'] = dfv['VehicleModel'].apply(lambda x: x[2:-2])
dfv['VehicleID'] = dfv['VehicleID'].apply(lambda x: x[2:-2])
dfv['VehicleMake'] = dfv['VehicleMake'].apply(lambda x: x[2:-2])
print(dfv.shape)
dfv.head()

(115344, 5)


,CustomerID,VehicleYOM,VehicleModel,VehicleID,VehicleMake
0,Cust20179,2011,Forrestor,Vehicle8898,BMW
1,Cust21384,2010,Malibu,Vehicle18688,Chevrolet
2,Cust33335,2001,Highlander,Vehicle8998,Toyota
3,Cust27118,1998,Neon,Vehicle28504,Saab
4,Cust13038,2000,Maxima,Vehicle30212,Toyota


In [13]:
dfv.isnull().sum().sum()

0

In [14]:
dfv.duplicated().sum()

86508

In [15]:
dfv.drop_duplicates(inplace=True)

In [16]:
dfv.shape

(28836, 5)

## Reading the Fraud Information data set

In [17]:
# Fraud Data : This Train.csv contains the Fraud information details
dfwt = pd.read_csv(r"C:\Users\Soorya\Documents\capstone-files\TrainData\Data\Traindata_with_Target.csv")
print(dfwt.shape)
dfwt.head()

(28836, 2)


,CustomerID,ReportedFraud
0,Cust20065,N
1,Cust37589,N
2,Cust24312,N
3,Cust5493,Y
4,Cust7704,Y


In [18]:
dfwt.isnull().sum().sum()

0

In [19]:
print(dfc.shape)
print(dfd.shape)
print(dfp.shape)
print(dfv.shape)
print(dfwt.shape)

(28836, 21)
(28836, 9)
(28836, 10)
(28836, 5)
(28836, 2)


In [20]:
# we need to perform merge on all the dataframes.
df = dfd.merge(dfp).merge(dfv).merge(dfc).merge(dfwt)

In [21]:
df.shape

(28836, 43)

In [22]:
df.head()

,CustomerID,InsuredAge,InsuredZipCode,InsuredGender,InsuredEducationLevel,InsuredOccupation,InsuredHobbies,CapitalGains,CapitalLoss,InsurancePolicyNumber,...,Witnesses,PoliceReport,AmountOfTotalClaim,AmountOfInjuryClaim,AmountOfPropertyClaim,AmountOfVehicleDamage,Year,Month,days,ReportedFraud
0,Cust10000,35,454776,MALE,JD,armed-forces,movies,56700,-48500,119121,...,0.0,NoReport,65501.0,13417,6071,46013,2015,2,3,N
1,Cust10001,36,454776,MALE,JD,tech-support,cross-fit,70600,-48500,119122,...,1.0,YES,61382.0,15560,5919,39903,2015,2,2,N
2,Cust10002,33,603260,MALE,JD,armed-forces,polo,66400,-63700,119123,...,3.0,NO,66755.0,11630,11630,43495,2015,1,15,N
3,Cust10003,36,474848,MALE,JD,armed-forces,polo,47900,-73400,119124,...,3.0,NO,66243.0,12003,12003,42237,2015,1,19,N
4,Cust10004,29,457942,FEMALE,High School,exec-managerial,dancing,0,-41500,119125,...,1.0,YES,53544.0,8829,7234,37481,2015,1,9,N


In [23]:
df.isnull().sum().sum()

0

In [24]:
# storing the file merged file as final_data.csv
df.to_csv("final_data.csv")